In [6]:
import pandas as pd
import os
import jive
import nibabel as nb
from glob import glob
from os.path import join
import nibabel as nb
import numpy as np

In [7]:
home = '/Users/lucindasisk/Box/LS_Folders/CMI_ABCD'
subids = pd.read_csv(home + '/ABCD_CMI_subids.csv', header=None)[0]
pheno_data = pd.read_csv(home + '/phenoData_analysis_08062019.csv', index_col='subjectkey')
gifti_data = join(home, 'ABCD_metrics')

In [8]:
#Read in Area data
data_dict_area=[]

def load_data_area(sublist, home_path):
    for sub in sublist:
        #Area data - Left func
        area_imgL = nb.load(join(gifti_data, 'area/{}.L.area.10k_fs_LR_sm_10.func.gii'.format(sub)))     
        area_img_dataL = [x.data for x in area_imgL.darrays]
        area_cur_dataL = np.reshape(area_img_dataL,(len(area_img_dataL[0]),len(area_img_dataL))).tolist()
        #Area data - Right func
        area_imgR = nb.load(join(gifti_data, 'area/{}.R.area.10k_fs_LR_sm_10.func.gii'.format(sub)))     
        area_img_dataR = [x.data for x in area_imgR.darrays]
        area_cur_dataR = np.reshape(area_img_dataR,(len(area_img_dataR[0]),len(area_img_dataR))).tolist()
        
        #Append data
        area_data = [sub]
        for x in range(0,len(area_cur_dataL)):
            dataL = area_cur_dataL[x]
            dataR = area_cur_dataR[x]
            area_data.append(dataL)
            area_data.append(dataR)
            
        data_dict_area.append(area_data)
    
    
    
    # Read in myelin data
data_dict_myelin=[]

def load_data_myelin(sublist, home_path):
    for sub in sublist:

        #Myelin Map data Left (2)
        myelin_imgL2 = nb.load(join(gifti_data, 'MyelinMap/{}.L.MyelinMap.10k_fs_LR_sm_10.func.gii'.format(sub)))     
        myelin_img_dataL2 = [x.data for x in myelin_imgL2.darrays]
        myelin_cur_dataL2 = np.reshape(myelin_img_dataL2,(len(myelin_img_dataL2[0]),len(myelin_img_dataL2))).tolist()
        #Myelin Map data Right(2)
        myelin_imgR2 = nb.load(join(gifti_data, 'MyelinMap/{}.R.MyelinMap.10k_fs_LR_sm_10.func.gii'.format(sub)))     
        myelin_img_dataR2 = [x.data for x in myelin_imgR2.darrays]
        myelin_cur_dataR2 = np.reshape(myelin_img_dataR2,(len(myelin_img_dataR2[0]),len(myelin_img_dataR2))).tolist()

        #Append data
        myelin_data = [sub]
        for x in range(0,len(myelin_cur_dataL2)):
            dataL = myelin_cur_dataL2[x]
            dataR = myelin_cur_dataR2[x]
            myelin_data.append(dataL)
            myelin_data.append(dataR)

        data_dict_myelin.append(myelin_data)

# Read in myelin data

data_dict_thick=[]

def load_data_thick(sublist, home_path):
    for sub in sublist:

        #Thickness data Left 2
        thickness_imgL2 = nb.load(join(gifti_data, 'thickness/{}.L.thickness.10k_fs_LR_sm_10.shape.gii'.format(sub)))     
        thickness_img_dataL2 = [x.data for x in thickness_imgL2.darrays]
        thickness_cur_dataL2 = np.reshape(thickness_img_dataL2,(len(thickness_img_dataL2[0]),len(thickness_img_dataL2))).tolist()
        #Thickness data Right 2
        thickness_imgR2 = nb.load(join(gifti_data, 'thickness/{}.R.thickness.10k_fs_LR_sm_10.shape.gii'.format(sub)))     
        thickness_img_dataR2 = [x.data for x in thickness_imgR2.darrays]
        thickness_cur_dataR2 = np.reshape(thickness_img_dataR2,(len(thickness_img_dataR2[0]),len(thickness_img_dataR2))).tolist()

        #Append data
        thick_data = [sub]
        for x in range(0,len(thickness_cur_dataL2)):
            dataL = thickness_cur_dataL2[x]
            dataR = thickness_cur_dataR2[x]
            thick_data.append(dataL)
            thick_data.append(dataR)

        data_dict_thick.append(thick_data)

In [9]:
#Run function, read in data from all subjects
#subids = subids[0:5]
load_data_area(subids, home)
load_data_myelin(subids, home)
load_data_thick(subids, home)

#### Reshape Data

In [10]:
## Reshape Area
cmi_df_area = pd.DataFrame(data_dict_area).rename(columns = {0:"subid",}).set_index('subid')
# Rename columns automatically
num_cols = len(cmi_df_area.columns)
left_cols_end = int(num_cols/2)
left_cols = range(1,left_cols_end+1)
right_cols = range(left_cols_end + 1, num_cols+1)
new_cols = ['area_L_' + str(i) for i in left_cols] + ['area_R_' + str(i) for i in right_cols]    
cmi_df_area.columns = new_cols[:num_cols]
for x in cmi_df_area.columns:
    cmi_df_area[x] = cmi_df_area[x].str.get(0)

In [11]:
cmi_df_myelin = pd.DataFrame(data_dict_myelin).rename(columns = {0:"subid",}).set_index('subid')
# Rename columns automatically
num_cols = len(cmi_df_myelin.columns)
left_cols_end = int(num_cols/2)
left_cols = range(1,left_cols_end+1)
right_cols = range(left_cols_end + 1, num_cols+1)
new_cols = ['myelin_L_' + str(i) for i in left_cols] + ['myelin_R_' + str(i) for i in right_cols]    
cmi_df_myelin.columns = new_cols[:num_cols]
cmi_df_myelin
for x in cmi_df_myelin.columns:
    cmi_df_myelin[x] = cmi_df_myelin[x].str.get(0)

In [12]:
cmi_df_thick = pd.DataFrame(data_dict_thick).rename(columns = {0:"subid",}).set_index('subid')
# Rename columns automatically
num_cols = len(cmi_df_thick.columns)
left_cols_end = int(num_cols/2)
left_cols = range(1,left_cols_end+1)
right_cols = range(left_cols_end + 1, num_cols+1)
new_cols = ['thickness_L_' + str(i) for i in left_cols] + ['thickness_R_' + str(i) for i in right_cols]    
cmi_df_thick.columns = new_cols[:num_cols]
for x in cmi_df_thick.columns:
    cmi_df_thick[x] = cmi_df_thick[x].str.get(0)

In [13]:
m1 = pd.merge(cmi_df_area, cmi_df_myelin, left_index = True, right_index = True, how='outer')
m2 =  pd.merge(m1, cmi_df_thick, left_index = True, right_index = True, how='outer')

In [19]:
#pheno_data = pheno_data.reset_index().rename(columns = {'subjectkey':'subjectid'}).set_index('subjectid')     
#combined = pd.merge(pheno_data, m2, left_index=True, right_index=True, how='outer')
m2.to_csv(join(home + '/CMI_Collab_Brain_Measures_1.4.19.csv'))